In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords  
import re
nltk.download('stopwords')  

[nltk_data] Downloading package stopwords to /Users/david/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
tweets_list1 = []

for i,tweet in enumerate(sntwitter.TwitterSearchScraper('CapitolRiots since:2021-01-05 until:2021-01-07').get_items()): #declare a username 
    if i>10: #number of tweets you want to scrape
        break
    tweets_list1.append([tweet.date, tweet.id, tweet.content, tweet.username]) #declare the attributes to be returned
    
# Creating a dataframe from the tweets list above 
tweets_df2 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

# def hashtag_extract(text):
#     l = []
#     mark_hash = 0
#     # for t in text:
#     #     print('t = ',t)
#     tag = ''
#     for i in range(len(text)):
#         print('t[i] = ',text[i])
#         if text[i] == '#': 
#             if tag != '': # if hashtag is right next to a previous hashtag
#                 print('Im here_1!')
#                 l.append(tag)
#                 print('l = ',l)
#                 tag = ''
#                 mark_hash = 0
#             else: #if hashtag not next to a previous hashtag
#                 print('Im here_2!')
#                 mark_hash = 1
#         elif mark_hash == 1 and (text[i]!=' ') and i != (len(text)-1) and text[i] != "\n": #if inside a hashtag and you're not at the end of the text or the end of a line
#             tag += text[i]
#             print('Im here_3!')
#             print('tag = ', tag)
#         elif mark_hash == 1 and (text[i] ==' ' or text[i] == "\n"): #if inside a hashtag and you're an empty space or the end of a line
#             print('Im here_4!')
#             l.append(tag)
#             #print('l = ',l)
#             tag = ''
#             mark_hash = 0
#         elif mark_hash == 1 and (i == (len(text) - 1)): #if inside a hashtag and you're not at the end of the text
#             tag += text[i]
#             l.append(tag)
#             tag = ''
#             mark_hash = 0
#         else:
#             tag = ''
#     return l

In [4]:
tweets_df2

,Datetime,Tweet Id,Text,Username
0,2021-01-06 23:59:53+00:00,1346969770243940352,"Dear generation Y &amp; Z,\n\nStop playing #Fo...",ZouhairSaddiki
1,2021-01-06 23:59:31+00:00,1346969679470813184,25th Amendment. Now.\n#CapitolHill #CapitolRio...,InSituMag
2,2021-01-06 23:58:52+00:00,1346969514680922113,Aye @CapitolPolice did you at least pinch the ...,MikeGrands
3,2021-01-06 23:58:46+00:00,1346969487736770563,@tedcruz so @tedcruz will you be resicinding y...,oldwomansage
4,2021-01-06 23:58:42+00:00,1346969473933332485,"""We had better plan now for an involuntary ext...",PamMHarris
5,2021-01-06 23:58:25+00:00,1346969403234127873,"If people can enter the #Capitol so easily, so...",Suresh_Gopalan
6,2021-01-06 23:58:25+00:00,1346969402286157831,Let’s call it for what it was today: terrorism...,SeanWasson
7,2021-01-06 23:58:11+00:00,1346969343242985474,Did the riots today remind anyone else of the ...,kriskline23
8,2021-01-06 23:57:47+00:00,1346969242013470720,#whiteprivilege #WhereIsTheLie #Amerika #Remov...,QueenG719
9,2021-01-06 23:57:26+00:00,1346969155002515456,#CapitolRiots #RacialJustice #BLM https://t.co...,givingyouthebi1


In [5]:
def clean_text(text, remove_stopwords = True):
    text = text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)
    text =  nltk.WordPunctTokenizer().tokenize(text)
    return text

def lemmatized_words(text):
    lemm = nltk.stem.WordNetLemmatizer()
    res = list(map(lambda word:
                                     list(map(lemm.lemmatize, word)),
                                     text))
    return res

In [16]:
bow_arr = []
tags_arr = []
bow_converter = CountVectorizer(tokenizer=lambda doc: doc, binary = True, min_df = 20, lowercase=False)
import os
my_dir = './data'
for item in os.listdir(my_dir):
    if os.path.isfile(os.path.join(my_dir, item)):
        print(os.path.join(my_dir, item))
        df = pd.read_csv(os.path.join(my_dir, item))
        df['Hashtags'] = df['Text'].apply(lambda x: re.findall(r"#(\w+)", x))
        lemma_tag = lemmatized_words(df['Hashtags'])
        df['text_cleaned'] = list(map(clean_text, df.Text))
        
        #lemmatized_words(df.text_cleaned)
        
#         if len(df['text_cleaned']) != 0:
#             bow = bow_converter.fit_transform(df['text_cleaned'])
#             words = bow_converter.get_feature_names()
#             #print('words = ',words)
#             bow_arr.append(bow)

        if lemma_tag:
            bow = bow_converter.fit_transform(lemma_tag)
            tags = bow_converter.get_feature_names()
            bow_arr.append(bow)
            tags_arr.append(tags)

print(bow_arr)

./data/scraped_tweets_#removetrumpnow_10000
0                                        [RemoveTrumpNow]
1                                        [RemoveTrumpNow]
2                                  [MAGA, RemoveTrumpNow]
3                            [SayHerName, RemoveTrumpNow]
4       [CoupAttempt, RemoveTrumpNow, ArrestTrump, Don...
                              ...                        
4996                                     [RemoveTrumpNow]
4997                                     [RemoveTrumpNow]
4998                       [ImpeachAgain, RemoveTrumpNow]
4999    [RemoveSeditionists, ImpeachTrumpAgainNOW, Rem...
5000                               [RemoveTrumpNow, coup]
Name: Hashtags, Length: 5001, dtype: object
./data/scraped_tweets_#impeachtrumpnow_10000
0                                      [ImpeachTrumpNow]
1                     [ImpeachTrumpNow, ImpeachPenceNow]
2                  [ImpeachTrumpNow, GOPBetrayedAmerica]
3                                      [ImpeachTrumpNow]
4

In [14]:
tags_arr[4]

['25thAmendment',
 '25thAmendmentNow',
 'ABD',
 'ABDdarbe',
 'AbdYanıyor',
 'America',
 'Amerika',
 'ArrestTrump',
 'BLM',
 'Biden',
 'BlackLivesMatter',
 'COVID19',
 'CapitalBuilding',
 'CapitalHill',
 'Capitol',
 'CapitolBuilding',
 'CapitolHill',
 'Capitole',
 'Capitolio',
 'Congress',
 'CoupAttempt',
 'DC',
 'DCProtests',
 'DomesticTerrorists',
 'DonaldTrump',
 'EEUU',
 'ElectoralCollege',
 'GOP',
 'ImpeachTrump',
 'ImpeachTrumpNow',
 'January6th',
 'MAGA',
 'MAGAIsCancelled',
 'Pence',
 'Sedition',
 'TREASON',
 'ThisIsAmerica',
 'Trump',
 'TrumpIsACriminal',
 'TrumpOut',
 'TrumpRally',
 'US',
 'USA',
 'USCapitol',
 'Washington',
 'WashingtonDC',
 'WhiteHouse',
 'america',
 'capitol',
 'capitolhill',
 'civilwar2021',
 'coup',
 'dcprotest',
 'democracy',
 'dolar',
 'insurrection',
 'treason',
 'trump',
 'usa',
 'washingtondc',
 'whiteprivilege',
 'καπιτωλιο']

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 977 bytes | 977.00 KiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
To https://github.com/hjkim63/aml_twitter_community.git
   68469fc..fa495d9  main -> main
